### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-74    3.7   91     0.31   0        Ruim     0.99      
-31    1.7   29     0.06   1        Bom      1.00      
-68    4.0   96     0.70   0        Ruim     1.00      
-51    1.5   17     0.25   1        Bom      0.99      
-46    1.1   17     0.38   1        Bom      0.98      
-80    7.4   92     0.65   0        Ruim     1.00      
-39    1.1   15     0.04   1        Bom      1.00      
-57    7.9   96     0.61   0        Ruim     1.00      
-67    3.9   57     0.25   0        Ruim     0.65      
-62    8.6   83     0.43   0        Ruim     1.00      
-49    2.7   33     0.28   1        Bom      0.97      
-62    8.1   80     0.52   0        Ruim     1.00      
-56    2.0   14     0.11   1        Bom      0.99      
-78    7.3   66     0.53   0        Ruim     1.00      
-54    1.9   7      0.34   1        Bom      0.98      
-71    8.7   46     0.85   0        Ruim     1.0

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
